# Royal Flush

An analysis of 1000 pairs of Poker hands to determine the winner. 

Author: Mark Richards

Email: mark.thomas.richards@outlook.com

## Overview

This is my attempt to solve [Project Euler Problem 54](https://projecteuler.net/problem=54) using data analysis. 

If you are not aware, Project Euler is an online archive of 800+ graded mathematically-oriented programming challenges, which are a good place to start when learning to solve reasonably complex problems.

Most data analysis case studies involve just manipulating data (grouping, aggregating, etc) but I wanted to try something a little more complex and as you will see below this is not a simple problem :)

## The Problem

We are given a 1000 line text file where each line represents a pair of 5-card poker hands. 

```
8C TS KC 9H 4S 7D 2S 5D 3S AC
5C AD 5D AC 9C 7C 5H 8D TD KS
3H 7H 6S KC JS QH TD JC 2D 8S
...
```

Each card is encoded as rank+suit: "8C" = Eight of clubs, etc.

The first five cards are player 1's hand and the second five are player 2's hand.

The goal is for us to determine, for each pair of hands, who was the winner, applying the standard ranking of poker hands, so that we can find out how many hands player 1 won.

Poker hand evaluation is quite complex. In order from lowest rank to highest we have:

- High Card: Highest value card.
- One Pair: Two cards of the same value.
- Two Pairs: Two different pairs.
- Three of a Kind: Three cards of the same value.
- Straight: All cards are consecutive values.
- Flush: All cards of the same suit.
- Full House: Three of a kind and a pair.
- Four of a Kind: Four cards of the same value.
- Straight Flush: All cards are consecutive values of same suit.
- Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.

So if player 1 has a Full House and player 2 has Three of a Kind, player 1 wins. But if both players have, say, Three of a Kind, then we have to apply additional rules to determine the winner (for Three of a Kind, the player with the highest card in the Three of a Kind). 

__You might reasonably ask what this has to do with data analysis:__ we are (probably) never going to be asked to analyse poker hands in real life. But we can imagine some analysis like this being needed in a real business task: for example, we could be analysing sales of a service to customers where the customer can choose different "modules" to be part of the service and perhaps the "value" for the company of selling different combinations of modules is different from simply the retail price - maybe some combinations correlate with longer customer retention, or with increased tech support revenue down the line, etc. So the company might present us with a ranked list of how to value sales of different combinations and this is not so different from a ranking of poker hands. 


## Loading the Data

Lets first read in the text file, adding some column names so we know what we are looking at.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
df_hands = pd.read_csv(
    "poker.txt", 
    sep=" ", 
    header=None, 
    names=[
        # player 1's hand
        "p1_1", "p1_2", "p1_3", "p1_4", "p1_5", 
        # player 2's hand
        "p2_1", "p2_2", "p2_3", "p2_4", "p2_5"]
)

df_hands

,p1_1,p1_2,p1_3,p1_4,p1_5,p2_1,p2_2,p2_3,p2_4,p2_5
0,8C,TS,KC,9H,4S,7D,2S,5D,3S,AC
1,5C,AD,5D,AC,9C,7C,5H,8D,TD,KS
2,3H,7H,6S,KC,JS,QH,TD,JC,2D,8S
3,TH,8H,5C,QS,TC,9H,4D,JC,KS,JS
4,7C,5H,KC,QH,JD,AS,KH,4C,AD,4S
...,...,...,...,...,...,...,...,...,...,...
995,3S,AD,9H,JC,6D,JD,AS,KH,6S,JH
996,AD,3D,TS,KS,7H,JH,2D,JS,QD,AC
997,9C,JD,7C,6D,TC,6H,6C,JC,3D,3S
998,QC,KC,3S,JC,KD,2C,8D,AH,QS,TS


## Cleaning

The data should already be clean since it is designed for this problem, but lets sense check it anyway.

Lets make sure we have no missing values.

In [3]:
df_hands.isna().sum()

p1_1    0
p1_2    0
p1_3    0
p1_4    0
p1_5    0
p2_1    0
p2_2    0
p2_3    0
p2_4    0
p2_5    0
dtype: int64

And make sure each field is only 2 characters.

In [4]:
df_hands.applymap(len).value_counts()

p1_1  p1_2  p1_3  p1_4  p1_5  p2_1  p2_2  p2_3  p2_4  p2_5
2     2     2     2     2     2     2     2     2     2       1000
dtype: int64

Finally, although its conceivable that we could get duplicated hands by chance, lets check for duplicated rows anyway.

In [5]:
df_hands.duplicated().sum()

0

## Analysis

Since we wil be adding more columns as we process the data, it makes sense to split the two players hands into two dataframes so we can see them better. 

We need to make sure that for each dataframe we add the current index to the table as a column, as we may end up manipulating the index later and we need to make sure we can still match up the players hands.

In [46]:
df_p1 = (df_hands
         .iloc[:, 0:5]
         .reset_index()
         .rename(columns={"index": "hand"})
        )

df_p2 = (df_hands
         .iloc[:, 5:]
         .reset_index()
         .rename(columns={"index": "hand"})
        )

In [47]:
df_p1

,hand,p1_1,p1_2,p1_3,p1_4,p1_5
0,0,8C,TS,KC,9H,4S
1,1,5C,AD,5D,AC,9C
2,2,3H,7H,6S,KC,JS
3,3,TH,8H,5C,QS,TC
4,4,7C,5H,KC,QH,JD
...,...,...,...,...,...,...
995,995,3S,AD,9H,JC,6D
996,996,AD,3D,TS,KS,7H
997,997,9C,JD,7C,6D,TC
998,998,QC,KC,3S,JC,KD


In [48]:
df_p2

,hand,p2_1,p2_2,p2_3,p2_4,p2_5
0,0,7D,2S,5D,3S,AC
1,1,7C,5H,8D,TD,KS
2,2,QH,TD,JC,2D,8S
3,3,9H,4D,JC,KS,JS
4,4,AS,KH,4C,AD,4S
...,...,...,...,...,...,...
995,995,JD,AS,KH,6S,JH
996,996,JH,2D,JS,QD,AC
997,997,6H,6C,JC,3D,3S
998,998,2C,8D,AH,QS,TS


__So, how are we going to solve this problem?__

Lets make a __plan__.

1. For each of the __poker hand ranking types__ (Three of a Kind, Straight, etc) we determine if a hand matches that type, then add a column to the dataframe for that type and populate it with True or False. For some of the hand rankings we need to know additional data (like for Three of A Kind, we need to know whether its three 9s or three 5s, etc) so for those cases we add an additional column to record that. 
2. Then we determine the players __best hand ranking__ (we can imagine that they might have Three of A Kind plus a Pair, so we need to know the _best_ of the different hand rankings they have). This can be populated in another column. 
3. For each hand, we compare whether player 1 or player 2 has the best hand, determine a __winner__, and populate a dataframe with which player won each hand.
4. Finally we can count __how many hands player 1 won__.

We need a simple way to parse each card string so that we know the suit and rank and also a simple way to get the ranks and suits of all the cards in a players hand. 

We'll convert the rank into integers so that they can easily be compared. We need to remember that 10 is represented as "T" and we also have to handle "J", "Q", "K", "A", which we will value as 11, 12, 13, 14 (aces are high in poker).

In [56]:
# Card Ranks
# ==========

def card_rank(card):
    """Returns the rank of a card as an integer."""
    
    card_num = card[0]
    
    switch = {
        "T": 10,
        "J": 11,
        "Q": 12,
        "K": 13,
        "A": 14
        }
    
    # if rank is T, J, Q, K, A return the matching integer
    if card_num in switch.keys():
        return switch[card_num]
    else:
        # convert number string to integer
        return int(card_num)
    
def get_hand_ranks(r):
    """
    Takes a row from the player dataframe and returns a 
    list of hand ranks.
    """
    return (r
            .iloc[1:6]
            .map(card_rank)
            .sort_values()
            .tolist()
           )
    
# Card Suits
# ==========
    
def card_suit(card):
    """Returns the suit of a card."""
    return card[1]

def get_hand_suits(r):
    """
    Takes a row from the player dataframe and returns a 
    list of hand suits.
    """
    return (r
            .iloc[1:6]
            .map(card_suit)
            .sort_values()
            .tolist()
           )

In [57]:
# testing
print(card_rank("TC"))
print(card_rank("6D"))
print(get_hand_ranks(df_p1.iloc[0, :]))

10
6
[4, 8, 9, 10, 13]


In [58]:
# testing
print(card_suit("TC"))
print(card_suit("6D"))
print(get_hand_suits(df_p1.iloc[0, :]))

C
D
['C', 'C', 'H', 'S', 'S']


#### Poker Hand Ranking Types

We need to go through the list of poker hand ranking types and for each come up with a function which takes a players dataframe of hands and returns the same dataframe with a boolean column added for whether each hand matches that type and also a column for the additional data we need to record.

Then if we apply all the functions in sequence on the players hand dataframe we will end up with an expanded dataframe which contains all the information to needed meet point (1) of our plan.

__Note:__ I have deliberately not refactored out some of the helper functions used below so that you can see exactly what is happening in each function just by looking at that function.

In [12]:
# High Card
# =========
#
# Every players hand will have a highest card, so we do not need to record whether
# this ranking type matches, we just need to record the highest card.

def high_card(df):
    """
    Takes a df of player hands and appends a new column containing
    the highest rank in each hand.
    """
    
    def max_of_hand(r):
        """Returns the highest rank in a hand."""
        return max(
            *get_hand_ranks(r)
        )
    
    df["high_card"] = df.apply(max_of_hand, axis=1)
    
    return df

In [13]:
# output
high_card(df_p1)

,hand,p1_1,p1_2,p1_3,p1_4,p1_5,high_card
0,0,8C,TS,KC,9H,4S,13
1,1,5C,AD,5D,AC,9C,14
2,2,3H,7H,6S,KC,JS,13
3,3,TH,8H,5C,QS,TC,12
4,4,7C,5H,KC,QH,JD,13
...,...,...,...,...,...,...,...
995,995,3S,AD,9H,JC,6D,14
996,996,AD,3D,TS,KS,7H,14
997,997,9C,JD,7C,6D,TC,11
998,998,QC,KC,3S,JC,KD,13


In [18]:
# One Pair
# ========
#
# Here we just need to determine if there is one duplicate rank
# in the hand and what rank is duplicated.

def one_pair(df):
    """
    Takes a df of player hands and appends a new column containing
    a boolean for if the hand contains a single pair and a column
    recording the rank of the pair.
    """          

    def find_pairs(r):
        """Find pairs in a hand."""
        # get the rank of each card in the hand
        cards = get_hand_ranks(r)

        # count the number of occurences of each rank
        rank_counts = {}
        for card in cards:
            if card in rank_counts.keys():
                rank_counts[card] = rank_counts[card] + 1
            else:
                rank_counts[card] = 1

        # find the number of ranks occuring exactly twice
        return [val for val in rank_counts.keys() if rank_counts[val] == 2]

    def pair_check(r):
        """
        Checks if a *single* pair is present in a hand and if so returns True.
        """
        pairs = find_pairs(r)
        
        # if there is only one pair, return True
        return len(pairs) == 1
    
    
    def pair_rank(r):
        """
        Checks if a *single* pair is present in a hand and if so returns its
        rank else 0.
        """
        pairs = find_pairs(r)
        
        # if there is only one pair, return True
        if len(pairs) == 1:
            return pairs[0]
        else:
            return 0
    
    # apply the functions to the dataframe
    df["one_pair"] = df.apply(pair_check, axis=1)
    df["one_pair_rank"] = df.apply(pair_rank, axis=1)
    
    return df

In [19]:
# output
one_pair(df_p1)

,hand,p1_1,p1_2,p1_3,p1_4,p1_5,high_card,one_pair,one_pair_rank
0,0,8C,TS,KC,9H,4S,13,False,0
1,1,5C,AD,5D,AC,9C,14,False,0
2,2,3H,7H,6S,KC,JS,13,False,0
3,3,TH,8H,5C,QS,TC,12,True,10
4,4,7C,5H,KC,QH,JD,13,False,0
...,...,...,...,...,...,...,...,...,...
995,995,3S,AD,9H,JC,6D,14,False,0
996,996,AD,3D,TS,KS,7H,14,False,0
997,997,9C,JD,7C,6D,TC,11,False,0
998,998,QC,KC,3S,JC,KD,13,True,13


In [33]:
# Two Pairs
# =========
#
# Here we need to determine if there are two separate pairs in the hand,
# add a boolean column, and add two more columns for the ranks of each pair.
# We can use much the same approach as for One Pair.


def two_pair(df):
    """
    Takes a df of player hands and appends a new column containing a 
    boolean for if two pairs exist, plus two columns for the ranks of
    each pair.
    """
    
    def find_two_pairs(r):
        """Find two pairs in a hand."""
        
        # get the rank of each card in the hand
        cards = get_hand_ranks(r)

        # count the number of occurences of each rank
        rank_counts = {}
        for card in cards:
            if card in rank_counts.keys():
                rank_counts[card] = rank_counts[card] + 1
            else:
                rank_counts[card] = 1
                
        # find the number of ranks occuring exactly twice
        return [val for val in rank_counts.keys() if rank_counts[val] == 2]
    
    def two_pair_check(r):
        """
        Checks if a *two* pairs is present in a hand and if so returns True.
        """
        pairs = find_two_pairs(r)
        
        # if there are two pairs, return True
        return len(pairs) == 2
    
    def two_pair_rank_1(r):
        """
        Checks if a *two* pairs is present in a hand and if so returns the
        highest ranked pair value.
        """
        pairs = find_two_pairs(r)
        
        # if there are two pairs return the max pair
        if len(pairs) == 2:
            return max(pairs)
        else:
            return 0
        
    def two_pair_rank_2(r):
        """
        Checks if a *two* pairs is present in a hand and if so returns the
        lowest ranked pair value.
        """
        pairs = find_two_pairs(r)

        # if there are two pairs return the min pair
        if len(pairs) == 2:
            return min(pairs)
        else:
            return 0
        
    # apply the functions to the dataframe
    df["two_pair"] = df.apply(two_pair_check, axis=1)
    df["two_pair_rank_1"] = df.apply(two_pair_rank_1, axis=1)
    df["two_pair_rank_2"] = df.apply(two_pair_rank_2, axis=1)
    
    
    return df
    

In [34]:
# output
(two_pair(df_p1)
 .query("two_pair == True")
 .head()
)

,hand,p1_1,p1_2,p1_3,p1_4,p1_5,high_card,one_pair,one_pair_rank,two_pair,two_pair_rank_1,two_pair_rank_2
1,1,5C,AD,5D,AC,9C,14,False,0,True,14,5
17,17,6S,8D,4C,8S,6C,8,False,0,True,8,6
43,43,2H,2C,6H,TH,TC,10,False,0,True,10,2
75,75,JC,6H,6S,JS,4H,11,False,0,True,11,6
111,111,5S,9C,KH,KD,9H,13,False,0,True,13,9


In [39]:
# Three of a Kind
# ===============
#
# Again we can use the same approach as above.
# There can only be one three of a kind in a hand,
# so we only need to record one additional rank.

def three_of_a_kind(df):
    """
    Takes a df of player hands and appends a new column containing a 
    boolean for if three of a kind exists, plus a column for the rank.
    """
    
    def find_three_of_a_kind(r):
        """Find three of a kind in a hand."""
        
        # get the rank of each card in the hand
        cards = get_hand_ranks(r)

        # count the number of occurences of each rank
        rank_counts = {}
        for card in cards:
            if card in rank_counts.keys():
                rank_counts[card] = rank_counts[card] + 1
            else:
                rank_counts[card] = 1
                
        # find the number of ranks occuring three times
        return [val for val in rank_counts.keys() if rank_counts[val] == 3]
    
    def three_of_a_kind_check(r):
        """
        Checks if three of a kind is present in a hand and if so returns True.
        """
        threes = find_three_of_a_kind(r)
        
        # if there is three of a kind return True
        return len(threes) == 1

    def three_of_a_kind_rank(r):
        """
        Checks if a three of a kind is present in a hand and if so returns the
        rank.
        """
        threes = find_three_of_a_kind(r)
        
        # if there are two pairs return the max pair
        if len(threes) == 1:
            return threes[0]
        else:
            return 0
        
    # apply the functions to the dataframe
    df["three_of_a_kind"] = df.apply(three_of_a_kind_check, axis=1)
    df["three_of_a_kind_rank"] = df.apply(three_of_a_kind_rank, axis=1)
    
    return df
    

In [38]:
# output
(three_of_a_kind(df_p1)
 .query("three_of_a_kind == True")
 .head()
)

,hand,p1_1,p1_2,p1_3,p1_4,p1_5,high_card,one_pair,one_pair_rank,two_pair,two_pair_rank_1,two_pair_rank_2,three_of_a_kind,three_of_a_kind_rank
52,52,AD,3C,3D,KS,3S,14,False,0,False,0,0,True,3
60,60,3D,AD,3C,3S,4C,14,False,0,False,0,0,True,3
189,189,QD,9H,5S,QS,QC,12,False,0,False,0,0,True,12
203,203,JD,JS,JC,TH,2D,11,False,0,False,0,0,True,11
472,472,6H,4H,6C,7H,6S,7,False,0,False,0,0,True,6


In [42]:
# Straight
# ========
# 
# For this we need to check if all the cards in a hand are consecutive values
# and if so what the highest card is.
# There are only a small number of possible straights, so we can just define a
# list of these and check for membership.

def straight(df):
    """
    Takes a df of player hands and appends a new column containing a 
    boolean for if a straight exists, plus a column for the rank.
    """
    
    possible_straights = [
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7],
        [4, 5, 6, 7, 8],
        [5, 6, 7, 8, 9],
        [6, 7, 8, 9, 10],
        [7, 8, 9, 10, 11],
        [8, 9, 10, 11, 12],
        [9, 10, 11, 12, 13],
        [10, 11, 12, 13, 14]
    ]
    
    def straight_check(r):
        """Returns true if straight in hand."""
        
        hand = get_hand_ranks(r)
        
        return hand in possible_straights
    
    def straight_rank(r):
        """Returns the highest rank of a straight in a hand."""
        
        hand = get_hand_ranks(r)
        
        if hand in possible_straights:
            return max(hand)
        else:
            return 0
        
    # apply the functions to the dataframe
    df["straight"] = df.apply(straight_check, axis=1)
    df["straight_rank"] = df.apply(straight_rank, axis=1)
    
    return df

In [59]:
# output
(straight(df_p1)
 .query("straight == True")
 .head()
)

,hand,p1_1,p1_2,p1_3,p1_4,p1_5,straight,straight_rank
6,6,6H,4H,5C,3H,2H,True,6
241,241,TS,8H,9S,6S,7S,True,10
575,575,3C,2D,4C,5S,6C,True,6
633,633,KS,JC,QD,TH,9S,True,13
